# <center> Introducción a las redes neuronales </center>
Las redes neuronales (también conocidas como sistemas conexionistas) son un modelo computacional basado en un gran conjunto de unidades neuronales simples (neuronas artificiales), de forma aproximadamente análoga al comportamiento observado en los axones de las neuronas en los cerebros biológicos.

![Red neuronal](https://s20.postimg.org/q0t1jzcxp/Colored_neural_network.png)

Las redes neuronales se componen de unas entradas (inputs), unas capas intermedias de procesamiento (hidden layers) y unas salidas (outputs). Cada conexión a un nodo está equipado con un peso, por el cual se multiplica el valor de entrada para evaluarlo dentro del nodo. 

 En la siguiente imagen podemos ver los pesos de las conexiones en una red neuronal muy simple. Esto daría como resultado una multiplicación en forma de matrix de la siguiente forma:
 
$$ \begin{bmatrix}w_{12} & w_{21}\\w_{12} & w_{22}\end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$$

Para más información sobre matrices consultar el [notebook sobre algebra lineal](https://github.com/mondeja/fullstack/blob/master/backend/src/001-matematicas/algebra_aritmetica/004-algebra_lineal/matrices.ipynb).

![Matrix red neuronal](https://s20.postimg.org/z50nbv3jx/matrix_neuronal.jpg)



Las redes neuronales son como los algoritmos, hay una caja negra con un número de entradas que produce unas salidas. Pero hay algo especial en esa caja que representa las redes neuronales: una serie de controles analógicos, algunos girando a la izquierda y otros girando a la derecha de tal modo que su giro afecta las salidas:

![caja redes neuronales](https://s20.postimg.org/kynkdr7lp/caja_negra_redes_neuronales.png)

La cuestión radica en encontrar los valores de esas manecillas que darán las salidas deseadas para todos los casos en nuestro problema a resolver (o al menos para la gran mayoría de casos). Ese reto puede dar lugar a sistemas de gran complejidad.

_________________________________________________________

## Perceptrones simples
Para dar inicio con las redes neuronales se parte de lo más simple: una neurona. Se le conoce como perceptrón simple. Se presenta, simplificadamente, así:

![perceptron simple](https://s20.postimg.org/e8734gxcd/perceptron_simple.png)

> En el [siguiente capítulo](https://github.com/mondeja/fullstack/blob/master/backend/src/analisis_de_datos/machine_learning/redes_neuronales/002-perceptron.ipynb) se toca el tema de los perceptrones en mayor profundidad, pero como puede ser bastante difícil de digerir así de primeras, empecemos con algo más intuitivo para entenderlo luego en profundidad.

### Implementando perceptrones simples en Python
 
Vamos a hacer que una neurona aprenda las tablas AND y OR de lógica booleana:

|  **AND**  | True  | False |
|-----------|:-----:|:-----:|
| **True**  | True  | False |
| **False** | False | False |

|  **OR**   | True  | False |
|-----------|:-----:|:-----:|
| **True**  | True  | True  |
| **False** | True  | False |

Llamaremos 1 a verdadero y 0 a Falso. Es una buena elección acostumbrarse a la notación numérica de estos valores ya que su manejo resulta más eficiente en las bibliotecas y con lenguajes a más bajo nivel como C son la única posibilidad.

#### Elementos básicos de una neurona artificial

Los elementos esenciales de una neurona son los siguientes:
- Entrada/s.
- Peso/s de las entrada/s.
- Función de activación.
- Umbral de dicha función.
- Salida/s.

Por cada entrada (0 ó 1 en este caso), la neurona empieza con un peso cuyo valor comienza al azar. El proceso que sigue es el siguiente:
1. Toma las entradas y multiplica cada una por su peso. Este proceso de multiplicación obliga al dato a tomar más o menos relevancia en el resultado, por eso se denomina **peso**. Todas las entradas pesadas se suman para obtener un valor, el cual se multiplica por el peso del umbral lo que provoca la ecuación para este caso: $E_1 \cdot P_1 + E_2 \cdot P_2 \cdot P_U$
2.  Este valor se pasa a la función de activación, en la cual será procesado para obtener un valor. El umbral es el cual se encarga de producir una salida binaria (o se activa o no se activa) dependiendo de si el valor que hemos obtenido lo supera o no.
3. Si la salida coincide con el resultado esperado, podríamos decir que ha aprendido, pero si no, los pesos se recalculan para seguir aprendiendo. Pero ¿cómo vamos a recalcular los pesos para que se inclinen hacia donde deseeamos para que en la próxima iteración el valor obtenido se asemeje más al objetivo? Podríamos recalcularlos de forma aleatoria, pero esto haría que el proceso de aprendizaje fuera ciego y más costoso. Para solventar este problema usamos la fórmula matemática de Frank Rossenblatt:
```
Error = Salida Esperada – Salida Real
Si Error != 0:
    Nuevo Peso (entrada 1) = Peso anterior (entrada 1) + tasa aprendizaje * Error * Entrada 1
    Nuevo Peso (entrada 2) = Peso anterior (entrada 2) + tasa aprendizaje * Error * Entrada 2
    Nuevo Peso (umbral) = Peso anterior (de la entrada del umbral) + tasa aprendizaje * Error
```
La tasa de aprendizaje será un número decimal positivo muy pequeño.

In [1]:
import random

class PerceptronSimple:
    """Perceptrón simple que aprende tablas de lógica booleana
    linealmente separables (activa el debug para observar su comportamiento
    en cada iteración)."""
    def __init__(self, tasa_aprendizaje, tabla, debug=False):
        self.TABLA = tabla
        self.entradas = [(regla[0], regla[1]) for regla in self.TABLA]   # Todas las entradas posibles
        
        """Iniciamos los pesos y el umbral en un número decimal 
        aleatorio comprendido entre -10 y 10"""
        self.PESOS = [random.uniform(-10, 10) for _ in range(3)] # Pesos para las dos entradas y el del umbral
        self.UMBRAL = random.uniform(-10, 10) # Valor inicial del umbral
        self.tasa_aprendizaje = tasa_aprendizaje
        
        self.n_iteracion = 0     # Vamos contando las iteraciones
        self.aprendiendo = True  # Bandera para indicar que está aprendiendo la tabla
        
        self._debug = debug
    
    def funcion_de_activacion(self, valor):
        """Función que devuelve 1 si el valor provisto
        supera el umbral, 0 en caso contrario."""
        return 1 if valor > self.UMBRAL else 0
    
    def salida_objetivo(self, entradas):
        """Obtiene la salida objetivo de la tabla de verdad
        según el valor de las entradas provistas"""
        for regla in self.TABLA:
            if entradas[0] == regla[0] and entradas[1] == regla[1]:
                return regla[2]
    
    def run(self):
        while self.aprendiendo:  # Mientras no haya aprendido la tabla
            self.n_iteracion += 1
            self.aprendiendo = False
            
            if self._debug:
                print("\n--------------------------------\nDEBUG: Iteración %d\n" % self.n_iteracion)
            
            for E in self.entradas:
                P = self.PESOS
                U = self.UMBRAL
                PU = self.PESOS[2]  # Peso del umbral 
                salida_real = self.funcion_de_activacion(E[0]*P[0] + E[1]*P[1] + PU)
                salida_objetivo = self.salida_objetivo(E)

                ERROR = salida_objetivo - salida_real
                if ERROR != 0:
                    P[0] = P[0] + self.tasa_aprendizaje * ERROR * E[0]
                    P[1] = P[1] + self.tasa_aprendizaje * ERROR * E[1]
                    P[2] = P[2] + self.tasa_aprendizaje * ERROR
                    self.aprendiendo = True
                    
                
                if self._debug:
                    print("Entradas = (%d, %d)" % (E[0], E[1]))
                    print("P_1 = %f\tP_2 = %f\tP_U = %f" % (P[0], P[1], PU))
                    print("UMBRAL = %f" % U)
                    print("Salida = %d\tObjetivo = %d\n" % (salida_real, salida_objetivo))
            if self._debug:            
                print("--------------------------------")
            
                
        
        print("Número de iteraciones necesarias: %d" % self.n_iteracion)
        

print("TABLA DE VERDAD AND")
TABLA_AND = [        # TABLA DE VERDAD AND
    [1, 1, 1],  # Verdadero y verdadero -> Verdadero
    [1, 0, 0],  # Verdadero y falso     -> Falso
    [0, 1, 0],  # Falso y verdadero     -> Falso
    [0, 0, 0]   # Falso y falso         -> Falso
]
                
ps = PerceptronSimple(0.3, TABLA_AND)
ps.run()

print("\n\n=======================================\n\n")

print("TABLA DE VERDAD OR")
TABLA_OR = [        # TABLA DE VERDAD OR
    [1, 1, 1],  # Verdadero y verdadero -> Verdadero
    [1, 0, 1],  # Verdadero y falso     -> Verdadero
    [0, 1, 1],  # Falso y verdadero     -> Verdadero
    [0, 0, 0]   # Falso y falso         -> Falso
]

ps = PerceptronSimple(0.3, TABLA_OR)
ps.run()

TABLA DE VERDAD AND
Número de iteraciones necesarias: 22




TABLA DE VERDAD OR
Número de iteraciones necesarias: 13


_________________________________


### <center>Siguiente capítulo: [**El perceptrón**](http://nbviewer.jupyter.org/github/mondeja/fullstack/blob/master/backend/src/analisis_de_datos/machine_learning/redes_neuronales/002-perceptron.ipynb) <center>
    
______________________________

> Fuentes:
- [Redes Neuronales parte 1 - Rafael Alberto Moreno Parra](https://openlibra.com/es/book/redes-neuronales-parte-1)
- [Mutlilayer perceptron (Part 1) - The nature of code](https://www.youtube.com/watch?v=u5GAVdLQyIg)